In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

## 1. 분석 데이터 준비

In [4]:
data = pd.read_csv('../breast-cancer-wisconsin.csv')
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [7]:
# 특성데이터 분할
X = data.loc[:,'Clump_Thickness':'Mitoses']
y = data.loc[:,'Class']

In [8]:
# 학습, 테스트 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify= y, random_state=42)

In [11]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2. 기본모델 적용

In [13]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.97265625

In [15]:
# 혼동행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('훈련데이터 오차행렬:\n',confusion_train)

훈련데이터 오차행렬:
 [[328   5]
 [  9 170]]


In [16]:
# 예측결과 레포트
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print('분류예측 레포트:\n', cfreport_train)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       333
           1       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [20]:
# 테스트 데이터
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9590643274853801

In [21]:
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  2  58]]


In [22]:
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



---

### 3. Grid Search

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
param = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

In [30]:
grid_search = GridSearchCV(LogisticRegression(), param, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             return_train_score=True)

In [31]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'C': 0.1}
Best Score: 0.9726
TestSet Score: 0.6491


---

### 4. Random Search

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [33]:
param_dist = {'C':randint(low=0.001, high=100)}

In [34]:
random_search = RandomizedSearchCV(LogisticRegression(), param_dist, cv=5, n_iter=100)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025529A95DC0>})

In [35]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'C': 14}
Best Score: 0.9745
TestSet Score: 0.9591
